Following this course: https://github.com/louthy/language-ext/wiki/Thinking-Functionally    

In [ ]:
// quick setup
#r "nuget:Shouldly"
#r "nuget:LanguageExt.Core"
using Shouldly;
using LanguageExt;
using static LanguageExt.Prelude;
using static LanguageExt.List;
using LanguageExt.ClassInstances;
using static System.Console;

Installed Packages LanguageExt.Core, 4.2.9 Shouldly, 4.0.3

### Starting simple
`AddOne :: int -> int`    
A function "maps" a value in the "domain" to a value in the "range". There is no "computation".

In [ ]:
int AddOne(int x) => x + 1;
return AddOne(5);

6

#### Important notes:    
`x` is not something that can change once we pass 5 in.    
This is not assignment - What is happening here is "binding".    
Once a domain value is "bound" is cannot be changed.     
`x` is a placeholder that can be referred to later, not changed.    
**There are no "variables", only values**

_side note, yes x is a variable being assigned a value - but it is forbidden 'functionaly'_

### Function Values
`Func<domain, range>`    

Similarly - the name `addOne` is just a "binding" to "the function that adds one to its input"    
"Every time you see the name `addOne`, replace it with the function that adds one to its input".    

For example, we can "bind" `addOne` to a new name.    
`plusOne` and `addOne` are "bound" to the same value, which is the function that adds one to its input.    
`plusOne :: int -> int`

In [ ]:
Func<int, int> plusOne = AddOne;
return (plusOne(5), AddOne(5));

Item1,Item2
6,6


### Simple Value
Imagine an operation that always returns `5` - this is a constant.    
in "C#" you can easily define that with the following: `const int C = 5;` or `static int C => 5; //Func<int>`.      
simple values do not need to be evaluated once "bound" - This is the subtle difference between them and function values, otherwise they are both values of Func.

With this, `5` and `five` are both just values that can be passed around. - This is a key part of thinking functionaly: functions are values that can be passed around to other functions.

A constant function for `5` would be defined as:    
`Func<Unit, int>`   
`five :: () -> int`

In [ ]:
const int C = 5;
//or
int C_alt => 5;
//or
Func<Unit, int> five = _ => 5;
//or
int Five(Unit _) => 5;

return (AddOne(C), AddOne(C_alt), AddOne(five(unit)), AddOne(Five(unit)));

// the presence of "Unit" is a bit annoying, this gets cleaned up later on.

Item1,Item2,Item3,Item4
6,6,6,6


### "Objects" vs "Values"

Values:
- A member of a domain. The domain of ints, strings, functions that map ints to strings, etc...    
- They are immutable
- They have no behaviour

Objects: 
- Encapsulation of data with associated behaviour
- Mutable

Avoid using "object" for standard values for functional C#.    
It should only be used to refer to true classes, or values that expose member functions.

### How types work with functions

Type notation: `domain -> range` == (C#) `Func<domain, range>`

"Higher-Order functions": A function that takes a function as a paramater, or returns another function.    
Think of LINQ `Select` or `Where`.

Example:
`EvalWith5ThenAdd2 :: (int -> int) -> int`

In [ ]:
int EvalWith5ThenAdd2(Func<int, int> fn) => 
    fn(5) + 2;

return EvalWith5ThenAdd2(AddOne);

8

In this function the domain is `(int -> int)` and the range is `int`.    
This means the input is a function that maps ints to ints, and the output is just an int.    

`timesThree :: int -> int`

In [ ]:
Func<int, int> timesThree = x => x * 3;

return EvalWith5ThenAdd2(timesThree);

17

Now, lets return a function rather than use one as input.    
`AdderGenerator :: int -> (int -> int)`

In [ ]:
Func<int, int> AdderGenerator(int numberToAdd) => 
    x =>  x + numberToAdd;

//or

Func<int, Func<int,int>> AdderGenerator_alt = numberToAdd => 
    x => x + numberToAdd;

// create new adders
var add1 = AdderGenerator(1);
var add2 = AdderGenerator_alt(2);

return (add1(1), add2(1));

Item1,Item2
2,3


### void?

How do we deal with something like:
```csharp
void PrintInt(int x)
{
    Console.WriteLine(x);
}
```
We cannot have `int -> void` since void is the absence of a value. But our function must have a range to map to.  Every function must have some output to map to.    
`Unit` to the rescue - it is like a `bool` which has two possible values `true`, `false`, though it only has one possible value `unit`.


In Funtional langauges the `Unit` type is often reffered to as `()` - however in C# this is a method invocation with no inputs.    
Since declaring this concept in c# fully is a bit verbose as we saw earlier with `Five` it is often omitted as an input.    

Example:
`WhatIsThis :: () -> ()`

In [ ]:
Unit WhatIsThis(Unit _) => unit;
// or better yet - which captures a bit of that functional syntax by omitting the unit input
Unit WhatIsThis() => unit;

// doesn't compile because we must return a unit
/*
Unit DoSomething() => 1 + 1; 
*/
Unit DoSomething() => ignore(1 + 1); // ignore is a wrapper to satisfy the c# compiler

#### Other useful types

In [ ]:
var tuples = ("Hello", 1);
Lst<int> list = List(1,2,3); // immutable, fixed size
Arr<int> arr = Array(1,2,3); // immutable, fixed size
Seq<int> seq = Seq(Range(1,10)); // similar to IEnumerable, but not infinite

Option<int> someInt = Some(1); // options wrap values that may be missing
Option<int> noInt = None;

### Currying
If 'functionaly' we can only have one input, how do we pass multiple inputs?    
1. tuples
2. currying! - a series of functions that each take one paramater    

Example: `PrintTwoParameters :: int -> int -> ()`

In [ ]:
Func<int, Func<int, Unit>> printTwoParameters = x => y => 
{
    Console.WriteLine($"x:{x}, y:{y}");
    return unit;
};

We now have a function that takes a single argument and returns a new function with that parameter baked in.  The new function also takes a single parameter.    

In [ ]:
var printOneThenY = printTwoParameters(1); // create a new func that prints 1 and whatever else you give it.
printOneThenY(2); // finish the function logic with the second parameter.
//or
printTwoParameters(1)(2);

x:1, y:2
x:1, y:2


Decalring functions the 'functional way' is a bit verbose and has a runtime cost in c#, so just declare them the 'classical way'

In [ ]:
static Unit PrintTwoParameters_classical(int x, int y)
{
    Console.WriteLine($"x={x}, y={y}");
    return unit;
}

PrintTwoParameters_classical(1,2);

x=1, y=2


Declaring functions "the classical way" misses out on the power of "curring"    
Language-ext has a prelude function to help restore this missing power.

In [ ]:
var printTwoParmeters_curry = curry<int, int, Unit>(PrintTwoParameters_classical);
var x = 1;
var y = 2;
var printOneThenY_curry = printTwoParmeters_curry(x);
var result  = printOneThenY_curry(y);
printTwoParmeters_curry(x)(y);

// or if input is a 'Func'
Func<int,int,Unit> printTwoParameters_classicalFunc = (x, y) => 
{
    Console.WriteLine($"x={x}, y={y}");
    return unit;
};
var alt = curry(printTwoParameters_classicalFunc); // input as 'Func' allows you to ommit type params

x=1, y=2
x=1, y=2


### Partial Application


Currying leads to 'partial application' - a powerful concept.    
The idea is to take a function of N parameters and apply some of the arguments, returning a new function of the remaining arguments.    
This is similar to `AdderGenerator` but more generalized.    

Example:    
`add :: int -> int -> int`    
partially apply 2 to get        
`addTwo :: int -> int`    

`isLessThan :: int -> int -> bool`    
partially apply 5 to get    
`isLessThan5 :: int -> bool`

In [ ]:
Func<int, int, int> add = (x, y) => x + y;
// partial application can be a bit messy in c# so language-ext helps out here.
Func<int,int> addTwo = par(add, 2); // partial application

WriteLine(addTwo(1));

Func<int, int, bool> isLessThan = (x, y) => x > y;
Func<int, bool> isLessThan5 = par(isLessThan, 5); 
Func<int, bool> isGreaterThan5 = lpar(isLessThan, 5);

WriteLine((isLessThan5(4), isGreaterThan5(4)));

// closer look at par and lpar
var isLessThan_fn = (int x) => (int y) => x > y; // a curry-able version of isLessThan, could also expressed as isLessThan_fn = curry(isLessThan);
Func<int, bool> isLessThan5_fn = isLessThan_fn(5); // partially apply 5
Func<int, bool> isGreaterThan5_fn = x => isLessThan_fn(x)(5); // partially apply 5 'from the left' - basically reverse par

WriteLine((isLessThan5_fn(4), isGreaterThan5_fn(4)));

// more examples


3
(True, False)
(True, False)


A few more examples    

Later on, we will revisit this example to show how much more powerful this can be.

In [ ]:
Func<Func<string, int, Unit>, int, int, int> adderWithPluggableLogger = (logger, x, y) =>  
{
    logger("x", x);
    logger("y", y);
    var result = x + y;
    logger("x+y", result);
    return result;
};

Func<string, int, Unit> consoleLogger = (name, val) => 
{
    Console.WriteLine($"{name}={val}"); 
    return unit;
};

var addWithConsoleLogging = par(adderWithPluggableLogger, consoleLogger);
var six = addWithConsoleLogging(2,4);

var addSixWithConsoleLogging = par(addWithConsoleLogging, six);
addSixWithConsoleLogging(3);

x=2
y=4
x+y=6
x=6
y=3
x+y=9


### Function Composition 
Taking the output of one function, and using it as the input of another function.    

For example, if you have a function that maps from T1 -> T2 and another function that maps from T2 -> T3, then you can compose them giving you a new function that maps from T1 -> T3.    

In [ ]:
// f(x)
Func<string, int> f = x => x.Length;
// g(x)
Func<int, bool> g = x => x < 3;

// a new function that uses the output of f(x) as the input of g(x)
// h(x) = g o f - that is g of f, or g after f
Func<string, bool> h = x => g(f(x));
//or using language-ext
Func<string, bool> h_composed = compose(f, g);
//language-ext alt
Func<string, bool> h_composed_alt = f.Compose(g);
//classical
bool h_classical (string x)
{
    var y = f(x);
    return g(y);
}


return h("hi");

True